Autor: Karol Szwed\
Data: 17.02.2023 r.\
Przedmiot: Kurs programowania w Python\
Projekt: Końcowe zadanie zaliczeniowe - analiza danych z użyciem API dla ZTM w Warszawie

# Cel projektu

Korzystając z danych dostępnych na stronie https://api.um.warszawa.pl/ zbierane są informacje o pozycjach autobusów w zadanym przedziale czasu. Do analizy wzięte zostały pod uwagę dwa okresy 1-godzinne, dokładnie od 13:30 do 14:30 w czwartek oraz od 16:30 do 17:30 w piątek.

Na podstawie zebranych danych została przeprowadzona analiza:
 - Średniej prędkości autobusów (np. ile autobusów przekroczyło prędkość 50 km/h?)
 - Punktualność autobusów w obserwowanym okresie (porównanie rzeczywistego czasu dojazdu na przystanki z rozkładem jazdy).
 
Następnie wyniki przeprowadzonej analizy zostały zwizualizowane oraz opisane.

# Import danych

Najpierw musimy pobrać wszystkie potrzebne dane, w tym:
- dane GPS autobusów w dwóch okresach 1-godzinnych (wczesne popołudnie oraz godzina szczytu),
- informacje o przystankach autobusowych;

Dodatkowo musimy utworzyć zbiory przystanków na trasie każdej z linii autobusowych.

## Pobranie danych GPS dla autobusów

Na początku pobrane zostały dane GPS dla autobusów, w tym:
- nr linii autobusowej
- szerokość i długość geograficzna
- czas pomiaru
- nr brygady
- nr pojadzu

In [ ]:
# Because we are working with large objects that are collected over long periods of time, let's define two helper
# functions that will save those objects as binary files, able to be quickly loaded later.

In [32]:
# All libraries used across the whole project
import warsaw_data_api
import datetime
import time
import json
import pandas as pd
import csv
import pickle
import my_pickle_save
from collections import defaultdict
import geopy.distance

"""
 * INPUT:
    - "pickle_filename" - string with the name of the pickle file where the object will be saved;
    - "obj_to_save" - the object that will be saved in the pickle file.
 * FUNCTION: Save an object to a pickle file.
 * OUTPUT: None; function has side effect of creating a pickle file with the saved object.
"""
def save_obj_as_pickle_file(pickle_filename, obj_to_save):
    with open(pickle_filename, 'wb') as f:
        pickle.dump(obj_to_save, f)
        

"""
 * INPUT:
    - "pickle_filename" - string with the name of the pickle file from which the object will be loaded;
    - "obj_to_load" - the object that will be loaded from the pickle file.
 * FUNCTION: Load an object from a pickle file.
 * OUTPUT: The object loaded from the pickle file.
"""
def load_obj_from_pickle_file(pickle_filename):
    with open(pickle_filename, 'rb') as f:
        return pickle.load(f)

Biblioteka "warsaw_data_api", dostępna poprzez PIP, udostępnia trzy funkcje, które wykorzystane zostały w projekcie:
- ztm.get_buses_location() -> zwraca dane GPS wszystkich autobusów w Warszawie
- ztm.get_lines_for_bus_stop_id(stop_id, stop_pole) -> zwraca listę linii zatrzymujących się na przystanku
- ztm.get_bus_stop_schedule_by_name("Banacha-Szpital", "01", "504") -> zwraca rozkład jazdy dla danego przystanku;

Uzasadnione jest to faktem, iż dokumentacja Warsaw Open Data API nie zawiera wszystkich zapytań API, jakie mogą zostać zażądane, ID żądań nie zawsze są aktualne oraz nie zawsze opisana jest odpowiedź na dane żądanie.\
Stąd w celu uniknięcia długiej serii prób i błędów uznałem, że skorzystam z tej prostej biblioteki open-source.

In [ ]:
"""
 * INPUT:
     - "filename" - string with name of the csv file where data will be written to;
     - "_MY_API_KEY" - string with my API key needed for API calls;
     - "timespan" - integer representing amount of seconds for how long the data will be collected;
     - "time_delta_for_buses" - maximum age (in seconds) of the bus GPS data to be considered valid;
     - "update_interval" - time (in seconds) between each data import from the Warsaw Open Data API.
 * FUNCTION: Gather live bus GPS data using API calls and save it to csv file (with header).
 * OUTPUT: None; function has side effect of creating csv file with bus GPS data.
"""
def import_bus_gps_data(filename, _MY_API_KEY, timespan, time_delta_for_buses, update_interval):
    ztm = warsaw_data_api.ztm(apikey=_MY_API_KEY)  # Pass API key
    start_time = time.time()

    print("Starting the data import at", datetime.datetime.now())
    print("Expected to end the data import at", datetime.datetime.now() + datetime.timedelta(seconds=timespan))

    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["lines", "latitude", "longitude", "time", "brigade", "vehicle_number"])  # write header

    while True:
        try:
            buses_all = ztm.get_buses_location()
            with open(filename, 'a', newline='') as file:
                writer = csv.writer(file)
                for bus in buses_all:
                    now = datetime.datetime.now()
                    time_diff = now - bus.time

                    # We want to gather data that is current, so we only collect location data
                    # that is at most 1 min old
                    if time_diff.seconds < time_delta_for_buses:
                        writer.writerow([bus.lines, bus.location.latitude, bus.location.longitude,
                                         bus.time.time(), bus.brigade, bus.vehicle_number])
            time.sleep(update_interval)  # wait for 1 minute between updates
            if time.time() - start_time > timespan:  # If timespan have passed, break the loop
                break
        except Exception:  # If an error occurs, ignore it and try again
            continue
            

# Example of usage:
filename = "Buses_location_afternoon.csv" # Where GPS data will be saved
_MY_API_KEY = "2620c061-1099-44d9-baab-fdc3a772ab29"  # My API key

timespan = 3600  # 60 minutes
time_delta_for_buses = 60  # 1 minute
update_interval = 60  # 1 minute

import_bus_gps_data(filename, _MY_API_KEY, timespan, time_delta_for_buses, update_interval)

# Pobranie danych o przystankach autobusowych

Po pobraniu pliku JSON ze strony API dokonujemy konwersji danych do formatu pandas DataFrame.

In [33]:
"""
 * INPUT:
    - "json_filename" - name of the JSON file containing bus stop data
 * FUNCTION: This function reads a JSON file containing bus stop data, extracts the necessary information, and
    transforms it into a pandas DataFrame. Renames the remaining columns for clarity.
 * OUTPUT: Returns a pandas DataFrame containing the bus stop data with columns: "stop_id", "stop_pole", "stop_name",
    "street_id", "latitude", "longitude", "direction".
"""
def extract_bus_stop_data(bus_stop_filename):
    json_file = open(bus_stop_filename, "r")

    # Load the JSON file
    json_dict = json.loads(json_file.read())

    # Extract the list of dictionaries
    dict_list = json_dict['result']

    # Convert each dictionary in the list
    new_dict_list = []
    for d in dict_list:
        new_dict = {item['key']: item['value'] for item in d['values']}
        new_dict_list.append(new_dict)

    # Convert the list of new dictionaries into a DataFrame
    bus_stops_table = pd.DataFrame(new_dict_list)

    # Drop the last column as it contains no useful information for us (route ID)
    last_column = bus_stops_table.columns[-1]
    bus_stops_table = bus_stops_table.drop(last_column, axis=1)

    # Rename the columns
    bus_stops_table.columns = ["stop_id", "stop_pole", "stop_name",
                               "street_id", "latitude", "longitude", "direction"]

    return bus_stops_table

In [34]:
# Example of usage for extract_bus_stop_data():
# _BUS_STOPS_JSON_FILENAME = "bus_stops.json"
# bus_stops_df = extract_bus_stop_data(_BUS_STOPS_JSON_FILENAME)
#
# print(bus_stops_df.head())
# print(bus_stops_df.shape)

## Tworzymy słownik zawierający zbiór z informacją o trasie każdej z linii

Aby uzyskać listę przystanków dla każdej linii autobusowej, tworzymy słownik. Klucze to numery linii, a wartości to listy tupli zawierających komplet informacji o przystanku autobusowym.

In [35]:
"""
 * INPUT:
    - "bus_stops_info" - a pandas DataFrame containing bus stop data.
    - "ztm" - an instance of the WarsawDataAPI class, used to make API calls to the Warsaw Open Data API.
 * FUNCTION: Iterates over each row in the "bus_stops_info", and for each bus stop, it retrieves the bus lines that stop
    there using the WarsawDataAPI. It then stores this information in a dictionary.
 * OUTPUT: Returns a dictionary where the keys are bus lines and the values are sets of tuples, each tuple representing
    a bus stop and containing the same data as a row in the "bus_stops_info" DataFrame.
"""
def create_dict_matching_bus_stops_to_lines(bus_stops_info, ztm):
    bus_line_stops_dict = {}
    for _, row in bus_stops_info.iterrows():
        stop_id = row['stop_id']
        stop_pole = row['stop_pole']
        stop_info = tuple(row.values)
        bus_lines = ztm.get_lines_for_bus_stop_id(stop_id, stop_pole)

        for bus_line in bus_lines:
            if bus_line not in bus_line_stops_dict:
                bus_line_stops_dict[bus_line] = set()
            bus_line_stops_dict[bus_line].add(stop_info)

    return bus_line_stops_dict

In [36]:
# Example of usage for create_dict_matching_bus_stops_to_lines():
# (!) First we have to run the extract_bus_stop_data() function to get the bus_stops_df DataFrame (!)
# _MY_API_KEY = "2620c061-1099-44d9-baab-fdc3a772ab29"  # My api key
# _ZTM = warsaw_data_api.ztm(apikey=_MY_API_KEY)  # Pass api key
# bus_line_stops = create_dict_matching_bus_stops_to_lines(bus_stops_df, _ZTM)
#
# print(bus_line_stops['504'])